In [438]:
import numpy as np
import scipy.linalg

import poly
import sym_qsp_opt
 
import matplotlib.pyplot as plt
import cudaq

In [439]:
# Classical Linear System Solver
class ClassicalSolver:
    def __init__(self, A, b):
        self.A = A
        self.b = b

    def solve(self):
        """
        Solves the linear system Ax = b using a classical solver.
        
        Returns:
            x: Solution vector.
        """
        x = scipy.linalg.solve(self.A, self.b)  # Using SciPy's solver for Ax = b
        return x

In [440]:
class QuantumCircuit:
    """Allows the quantum circuit to input data, output expectation values
    and calculate gradients of variational parameters via finite difference"""

    def __init__(self, U_A, U_b):
        """Define the quantum circuit in CUDA Quantum"""
        @cudaq.kernel
        def kernel(U_A:list[float], U_b:list[float]):
            qubits = cudaq.qvector(2)

            L=U
            G=B
            h(qubits[0])

            mz(qubits)

        self.kernel = kernel

    def run(self, U_A, U_b):
        result = cudaq.sample(self.kernel, U_A, U_b)
        #print(cudaq.draw(self.kernel, U_A, U_b))
        return result

In [441]:
class QuantumSolverQSVT:
    def __init__(self, A, b, epsilon=1e-2, phases=None):
        """
        Initializes the QuantumSolverQSVT with matrix A and vector b.
        
        Parameters:
            A (numpy.ndarray): The coefficient matrix for the linear system.
            b (numpy.ndarray): The right-hand side vector of the linear system.
        """
        self.A = A
        self.b = b
        # TODO: alpha and kappa may need to be replaced by their respective upper bounds
        self.alpha = np.linalg.norm(A, ord=2)  # Spectral norm
        self.A_rescaled = self.A / self.alpha
        self.kappa = np.linalg.cond(A) # Condition number
        self.b_norm = np.linalg.norm(b) # Normalization 
        self.epsilon = epsilon # Desired presision
        self.N = len(b)
        self.n = int(np.log2(self.N))
        self.m = 1 # Number of ancillas for the blocj-encoding of A

        if phases is None:
            pg = poly.PolyOneOverX()
            pcoefs, scale = pg.generate(kappa=self.kappa, 
                                        epsilon=self.epsilon, 
                                        chebyshev_basis=True, 
                                        return_scale=True)
            # Using odd parity and instantiating desired coefficeints.
            parity = 1
            coef = pcoefs[parity::2]
            # Optimize to the desired function using Newton solver.
            crit = 1e-12
            (phases, err, total_iter, qsp_seq_opt) = sym_qsp_opt.newton_solver(coef, parity, crit=crit)
        self.phases = phases    

    def _block_encode_A(self):
        """
        Creates a block-encoded unitary matrix U that embeds A / alpha in its top-left block.
        """
        I = np.eye(self.A_rescaled.shape[0])
        top_right_block = scipy.linalg.sqrtm(I - self.A_rescaled @ self.A_rescaled.T.conj())
        bottom_left_block = scipy.linalg.sqrtm(I - self.A_rescaled.T.conj() @ self.A_rescaled)

        U = np.block([[self.A_rescaled, top_right_block],
                      [bottom_left_block, -1*self.A_rescaled.T.conj()]])
        return U

    def _state_preparation_b(self):
        """
        Creates a state-preparation unitary matrix Q s.t. Q|0> = |b> / ||b>|
        """
        B = np.column_stack((self.b / self.b_norm, np.random.randn(self.N, self.N - 1)))
        # Apply QR decomposition to B to get an orthonormal basis
        # The Q matrix from the QR decomposition will be unitary, and the first column will be b
        Q, _ = scipy.linalg.qr(B, mode='economic')
        return Q

    def Pi_phi(self,k):
        """
        Input:
        angles phi: vector of dim d/2 where d is the degree of poly that approximate 1/x

        Output:
        Unitary matrix Pi_phi=e^(i*phi*(2*Pi-I)) (eq 27)

        """
        diagonal= -1*self.phases[k]
        diagonal[0]=diagonal[0]*-1
        exp_diagonal = np.exp(1j * diagonal)

        # Create the matrix with np.diag()
        Pi = np.diag(exp_diagonal)

        return Pi

    def _P(self, phi):
        dim = int(2**self.m)
        P = np.diag([np.exp(-1j*phi)] + [np.exp(1j*phi)]*(dim-1))
        return P
    
    def solve(self):
        """
        Solves the system Ax = b using a QSVT-based inversion (placeholder).
        
        Returns:
            x (numpy.ndarray): Solution vector, as if from a QSVT inversion.
        """

        num_qubits = self.n + self.m
        U_b = self._state_preparation_b()
        U_A = self._block_encode_A()

        Pi=np.zeros((len(self.phases), 2**self.n, 2**self.n))
        i=0
        for phi in self.phases[1:]:
            Pi[i,:,:]=self._P(phi)
            i=i+1 

            
        print(U_b.shape)
        print(U_A.shape)
        
        print("create obj")
        quantum_circuit = QuantumCircuit(U_A, U_b)
        #print(cudaq.draw(quantum_circuit.kernel,U_A))
        print("run circuit")
        x = quantum_circuit.run(U_A, U_b)

        return x

In [442]:
# Define a test matrix A and vector b
A = np.array([[4, 1], [1, 3]], dtype=float)
b = np.array([[1, 2],[1,3]], dtype=float)

solver = QuantumSolverQSVT(A, b)

x = solver.solve()

print(x[0])



b=19, j0=13
[PolyOneOverX] minimum [-2.82888078] is at [-0.25849557]: normalizing
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 1000.
iter: 001 --- err: 1.950e-01
iter: 002 --- err: 2.575e-02
iter: 003 --- err: 9.790e-04


iter: 004 --- err: 1.707e-06
iter: 005 --- err: 5.266e-12
iter: 006 --- err: 1.169e-15
[sym_qsp] Stop criteria satisfied.
(2, 2)
(4, 4)
create obj
run circuit


/tmp/ipykernel_1701/3476029309.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  Pi[i,:,:]=self._P(phi)


IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
u = solver._block_encode_A()

In [ ]:
u.T.conj() @ u

array([[ 1.00000000e+00,  3.40761042e-17, -6.25890399e-18,
        -5.62634324e-17],
       [ 3.40761042e-17,  1.00000000e+00,  3.20364067e-17,
        -1.10208052e-17],
       [-6.25890399e-18,  3.20364067e-17,  1.00000000e+00,
         3.82747757e-17],
       [-5.62634324e-17, -1.10208052e-17,  3.82747757e-17,
         1.00000000e+00]])

In [ ]:


# Initialize the QSVT-based quantum solver with block encoding
quantum_solver = QuantumSolverQSVT(A, b)

# Solve the system
quantum_solution = quantum_solver.solve()
print("Solution vector x (QuantumSolverQSVT):", quantum_solution)

b=19, j0=13
[PolyOneOverX] minimum [-2.82888078] is at [-0.25849557]: normalizing
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 1000.
iter: 001 --- err: 1.950e-01
iter: 002 --- err: 2.575e-02
iter: 003 --- err: 9.790e-04
iter: 004 --- err: 1.707e-06
iter: 005 --- err: 5.266e-12
iter: 006 --- err: 1.169e-15
[sym_qsp] Stop criteria satisfied.
(2, 2)
(4, 4)
here
here


/tmp/ipykernel_1701/2744258743.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  Pi[i,:,:]=self._P(phi)


Solution vector x (QuantumSolverQSVT): [<cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.SampleResult object at 0x7f49c1cd7630>, <cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.SampleResult object at 0x7f49c024fe70>, <cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.SampleResult object at 0x7f49c2d6b830>, <cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.SampleResult object at 0x7f49c1d2aa70>]


In [ ]:
# new code